In [1]:
# import the required libraries
import numpy as np
import pandas as pd
import re
from pandas import read_csv
# set the path to the data files
pcard2014_name = 'Analytics_mindset_case_studies_PCard_FY2014.csv'
pcard2015_name = 'Analytics_mindset_case_studies_PCard_FY2015.csv'

# set desired column names
names = {
    'AgencyNumber': 'AgencyNumber',
    'AgencyName': 'AgencyName',
    'CardholderLastName': 'CardholderLastName',
    'CardholderFirstInitial': 'CardholderFirstInitial',
    'Description': 'Description',
    'Amount': 'Amount',
    'Vendor': 'Vendor',
    'TransactionDate': 'TransactionDate',
    'PostedDate': 'PostedDate',
    'MCC': 'MCC'
}

# read the data files into pandas dataframes
pcard2014 = read_csv(pcard2014_name, header=0, names=names, encoding='UTF-8')
pcard2015 = read_csv(pcard2015_name, header=0, names=names, encoding='UTF-8')

/var/folders/v0/hpm7rd7j4n39wr93kp2m5xwm0000gn/T/ipykernel_19376/2046240363.py:26: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  pcard2015 = read_csv(pcard2015_name, header=0, names=names, encoding='UTF-8')


In [2]:
# Concatenating the two dataframes
pcard = pd.concat([pcard2014, pcard2015])

In [3]:
# Resetting the index to avoid duplicate index values
# drop=True avoids the old index being added as a column
# inplace=True modifies the DF in place without creating a new object
pcard.reset_index(drop=True, inplace=True) 

In [4]:
pcard.shape

(880037, 10)

In [5]:
pcard.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 880037 entries, 0 to 880036
Data columns (total 10 columns):
 #   Column                  Non-Null Count   Dtype 
---  ------                  --------------   ----- 
 0   AgencyNumber            880037 non-null  int64 
 1   AgencyName              880037 non-null  object
 2   CardholderLastName      880037 non-null  object
 3   CardholderFirstInitial  880037 non-null  object
 4   Description             880037 non-null  object
 5   Amount                  880037 non-null  object
 6   Vendor                  880037 non-null  object
 7   TransactionDate         657156 non-null  object
 8   PostedDate              657156 non-null  object
 9   MCC                     880037 non-null  object
dtypes: int64(1), object(9)
memory usage: 67.1+ MB


## Task 3: Data  Preprocessing (Extract, Transform, and Load)

### Information Schema

| Column Name             | Data Type   | Description                       |
|--------------------------|-------------|-----------------------------------|
| **AgencyNum**           | `int`       | Unique identifier for the agency |
| **AgencyName**          | `str`       | Name of the agency               |
| **CardholderLastName**  | `str`       | Last name of the cardholder      |
| **CardholderFirstInitial** | `str`    | First initial of the cardholder  |
| **Description**         | `str`       | Description of the transaction   |
| **Amount**              | `float`     | Transaction amount               |
| **Vendor**              | `str`       | Vendor associated with the transaction |
| **TransactionDate**     | `datetime`  | Date the transaction occurred    |
| **PostedDate**          | `datetime`  | Date the transaction was posted  |
| **MCC**                 | `str`       | Merchant Category Code           |


### Tranforming TransactionDate and PostedDate

In [6]:
# Transforming the TransactionDate and PostedDate to ISO datetime format
pcard['TransactionDate'] = pd.to_datetime(pcard['TransactionDate'], format='%m/%d/%Y %H:%M')
pcard['PostedDate'] = pd.to_datetime(pcard['PostedDate'], format='%m/%d/%Y %H:%M')

### Transforming Amount

In [7]:
# Parsing amount to String
pcard['Amount'] = pcard['Amount'].astype(str)
pcard['Amount'] = pcard['Amount'].str.replace('$', '') # Removing $ from Amount
pcard['Amount'] = pcard['Amount'].str.replace(',', '') # Removing , from Amount

# Changing () to - in Amount
pcard['Amount'] = pcard['Amount'].str.replace('(', '-') 
pcard['Amount'] = pcard['Amount'].str.replace(')', '')

# Parsing Amount back to float
pcard['Amount'] = pcard.Amount.astype(float)

### Mapping AgencyNumber to correct AgencyName

In [8]:
# Find AgencyNumber with multiple AgencyName
mult_agency = pcard.groupby('AgencyNumber')['AgencyName'].nunique()
mult_agency[mult_agency > 1]

AgencyNumber
4000     3
19000    2
21500    2
26500    2
30900    2
34000    2
34200    2
50900    2
58800    2
60600    2
Name: AgencyName, dtype: int64

In [9]:
# Creating mapping table for multiple AgencyNames
agency_mapping_table = {
    4000: 'DEPARTMENT OF AGRICULTURE FOOD & FORESTRY',
    19000: 'OKLAHOMA STATE BOARD OF COSMETOLOGY AND BARBERING',
    21500: 'OLKAHOMA STATE BOARD OF DENTISTRY',
    26500: 'DEPARTMENT OF EDUCATION',
    30900: 'DEPARTMENT OF EMERGENCY MANAGEMENT',
    34000: 'OKLAHOMA STATE DEPARTMENT OF HEALTH',
    34200: 'OFFICE OF THE CHIEF MEDICAL EXAMINER',
    50900: 'BOARD OF EXAM. FOR LT CARE ADMIN',
    58800: 'OKLAHOMA REAL ESTATE COMMISSION',
    60600: 'UNIVERSITY CENTER OF SOUTHERN OKLAHOMA'
}

# Replacing duplicate AgencyNames with correct AgencyName
# .map() replaces the values in pcard['AgencyName'] with the values in agency_mapping_table
# .fillna() fills in the values that are not in the mapping table with the original values
pcard['AgencyName'] = pcard['AgencyNumber'].map(agency_mapping_table).fillna(pcard['AgencyName'])

In [10]:
# Check for multiples after transformation
mult_agency = pcard.groupby('AgencyNumber')['AgencyName'].nunique()
mult_agency[mult_agency > 1]

Series([], Name: AgencyName, dtype: int64)

### Fix general string formatting

In [11]:
# Remove all leading & trailing blanks

for col in pcard.select_dtypes(include='object'):
    pcard[col] = pcard[col].str.strip()   

# Capitalize all strings

for col in pcard.select_dtypes(include='object'):
    pcard[col] = pcard[col].str.upper()

In [12]:
pcard.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 880037 entries, 0 to 880036
Data columns (total 10 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   AgencyNumber            880037 non-null  int64         
 1   AgencyName              880037 non-null  object        
 2   CardholderLastName      880037 non-null  object        
 3   CardholderFirstInitial  880037 non-null  object        
 4   Description             880037 non-null  object        
 5   Amount                  880037 non-null  float64       
 6   Vendor                  880037 non-null  object        
 7   TransactionDate         657156 non-null  datetime64[ns]
 8   PostedDate              657156 non-null  datetime64[ns]
 9   MCC                     880037 non-null  object        
dtypes: datetime64[ns](2), float64(1), int64(1), object(6)
memory usage: 67.1+ MB


### Filtering Data

**Necessary Filters:**
1. Remove NA Values
2. Filter Agency for only OSU transactions
3. Filter TransactionDate for 2014 calendar year transactions


In [13]:
# Step 1: Remove all rows with missing values
pcard_filtered = pcard.dropna()

In [14]:
# Step 2: Filter Agency to only include OSU transactions
pcard_filtered = pcard_filtered[pcard_filtered['AgencyNumber'] == 1000]

In [15]:
# Step 3: Filter TransactionDate for 2014 calendar year only
pcard_filtered = pcard_filtered[pcard_filtered['TransactionDate'].dt.year == 2014]

In [16]:
# Reindex the dataframe
pcard_filtered = pcard_filtered.reset_index(drop=True)

In [17]:
# Check for number of rows in filtered data
pcard_filtered.shape

(116031, 10)

In [18]:
# Check total amount spent in filtered data
print(pcard_filtered['Amount'].sum())

33504148.340000004


In [19]:
# Save the filtered data to a new CSV file
pcard_filtered.to_csv('pcard_filtered.csv', index=False)

### Part 3 Final Output

- **Number of Rows**: `116031`
- **Total $ Amount**: `$33504148.34`


## Task 5: Analyzing the Data

### Internal Control Tests

| **Internal Control**                                | **Test to Perform**                                                                                                                                       |
|-----------------------------------------------------|-----------------------------------------------------------------------------------------------------------------------------------------------------------|
| 1. No more than $5,000 per transaction.             | List transactions in 2014 over $5,000 with details (`Amount`, `CardholderLastName`, `Description`, `Vendor`, `TransactionDate`, `PostedDate`, `MCC`). Sort by amount in decreasing order.    |
| 2. No more than $50,000 per year.                   | List employees who spent more than $50,000 in 2014 with total amounts. Sort by total amount spent in decreasing order                                                       |
| 3. No more than $10,000 per month without approval. | List employees who spent more than $10,000 per month in 2014 with total amounts. Sort by month and total amount spent by decreasing order of amounts                                    |
| 4. No splitting over $5,000 on the same day.        | List transactions where the same purchaser split purchases over $5,000 with the same vendor on the same day. Sort by `TransactionDate` in ascending order                  |
| 5. Count split purchases.                           | Count how many times each purchaser split purchases over $5,000 on the same day. Sort by count in descending decreasing order                                                            |
| 6. Splits across cardholders (Optional).            | List vendors with combined totals over $5,000 across multiple cardholders on the same day. Sort by `TransactionDate` in ascending order                                    |
| 7. No food/mileage while traveling.                 | List transactions with `hotel`, `motel`, `resort`, or `inn` MCCs that also include `food` or `restaurant` MCCs on the same day. Sort in ascending order by name and then amount.   |


#### 1. Filter for individual transactions over $5000

In [20]:
# Filter transactions where Amount > $5000
transactions_over_5000 = pcard_filtered[pcard_filtered['Amount'] > 5000]

In [21]:
# Select relevant columns and sort by Amount in descending order
transactions_over_5000 = transactions_over_5000[['CardholderLastName', 'CardholderFirstInitial', 'Description', 'Amount', 'Vendor', 'TransactionDate', 'PostedDate', 'MCC']].sort_values('Amount', ascending=False)
transactions_over_5000 = transactions_over_5000.reset_index(drop=True)
transactions_over_5000 

,CardholderLastName,CardholderFirstInitial,Description,Amount,Vendor,TransactionDate,PostedDate,MCC
0,CLARK,C,GENERAL PURCHASE,29731.61,OEC OTIS ELEVATOR CO,2014-08-18,2014-08-19,DIRCT MARKETING/DIRCT MARKETERS--NOT ELSEWHERE...
1,ROPERS,A,ROOM CHARGES,27657.55,WINGATE MANHATTAN,2014-05-22,2014-05-23,"LODGING--HOTELS,MOTELS,AND RESORTS"
2,ROPERS,A,ROOM CHARGES,22764.84,HAMPTON INN & SUITES CRN,2014-05-23,2014-05-26,HAMPTON INNS
3,ROPERS,A,GENERAL PURCHASE,21243.93,MAGDALENE COLLEGE,2014-09-08,2014-09-09,BUSINESS SERVICES NOT ELSEWHERE CLASSIFIED
4,STOVER,C,GENERAL PURCHASE,19233.28,COCKRELL EYECARE CENTER,2014-12-24,2014-12-26,OPTICIANS AND DISPENSING
5,ROPERS,A,AIR TRAVEL,16517.20,SOUTHWES 5262393866444,2014-08-28,2014-09-01,SOUTHWEST AIRLINES
6,WILLIAMS,J,ROOM CHARGES,15327.36,HILTON BOSTON DOWNTOWN,2014-09-24,2014-09-26,HILTON HOTELS
7,BILBEISI,S,ROOM CHARGES,13186.64,ROWN NYC FD,2014-08-01,2014-08-04,"LODGING--HOTELS,MOTELS,AND RESORTS"
8,CLARK,C,GENERAL PURCHASE,12600.00,TLC PAINTING,2014-01-24,2014-01-27,SPECIAL TRADE CONTRACTORS--NOT ELSEWHERE CLASS...
9,CLARK,C,GENERAL PURCHASE,12380.00,AABB-AMER. ASSO. BLOOD,2014-04-04,2014-04-07,MEMBERSHIP ORGANIZATIONS--NOT ELSEWHERE CLASSI...


In [22]:
print(transactions_over_5000['Amount'].sum())
print(transactions_over_5000.shape)
print(transactions_over_5000.dtypes)

357870.91000000003
(33, 8)
CardholderLastName                object
CardholderFirstInitial            object
Description                       object
Amount                           float64
Vendor                            object
TransactionDate           datetime64[ns]
PostedDate                datetime64[ns]
MCC                               object
dtype: object


#### 2. Filter for total annual expenditure over $50,000

In [23]:
# Group by CardholderLastName and CardholderFirstInitial and aggregate annual spending
cardholder_annual_spending = pcard_filtered.groupby(['CardholderLastName', 'CardholderFirstInitial'])['Amount'].sum()
cardholder_annual_spending = cardholder_annual_spending.to_frame().reset_index()

In [24]:
# Filter Cardholders with annual spending over $50,000 and sort by Amount in descending order
spending_over_50000 = cardholder_annual_spending[cardholder_annual_spending['Amount'] > 50000].sort_values('Amount', ascending=False)
spending_over_50000.reset_index(drop=True, inplace=True)
spending_over_50000

,CardholderLastName,CardholderFirstInitial,Amount
0,HINES,G,1595302.32
1,TORNAKIAN,M,1182094.86
2,HEUSEL,J,627608.56
3,FITZPATRICK,S,437479.84
4,BOWERS,R,392761.34
...,...,...,...
125,GRUNTMEIR,J,50553.45
126,HUFFSTETLER,N,50352.46
127,WOODWARD,K,50318.47
128,WILLIAMS,G,50297.55


In [25]:
print(spending_over_50000['Amount'].sum())
print(spending_over_50000.shape)
print(spending_over_50000.dtypes)

15723630.55
(130, 3)
CardholderLastName         object
CardholderFirstInitial     object
Amount                    float64
dtype: object


#### 3. Filter for monthly expenditures over $10,000 and sort by month

In [26]:
# Group by CardholderLastName, CardholderFirstInitial, and Month and aggregate monthly spending
monthly_spending = (
    pcard_filtered.groupby([
        'CardholderLastName',
        'CardholderFirstInitial',
        pcard_filtered['TransactionDate'].dt.month # Extracts the month directly from TransactionDate without having to create a new column
    ])['Amount'].sum().reset_index()
)

# Rename TransactionDate column to Month for clarity
monthly_spending.rename(columns={'TransactionDate': 'Month'}, inplace=True)

In [27]:
# Filter for Cardholders with monthly spending over $10,000
monthly_spending_over_10000 = monthly_spending[monthly_spending['Amount'] > 10000]

In [28]:
# Sort by Month (ascending) and Amount (descending)
monthly_spending_over_10000 = monthly_spending_over_10000.sort_values(
    ['Month', 'Amount'], ascending=[True, False]
)

# Dictionary to map month numbers to month names
month_order = {
    1: 'January', 2: 'February', 3: 'March', 4: 'April', 5: 'May', 6: 'June', 7: 'July', 8: 'August', 9: 'September', 10: 'October', 11: 'November', 12: 'December'
}

# Remap month numbers to month names
monthly_spending_over_10000['Month'] = monthly_spending_over_10000['Month'].map(month_order)
monthly_spending_over_10000 = monthly_spending_over_10000.reset_index(drop=True)
monthly_spending_over_10000

,CardholderLastName,CardholderFirstInitial,Month,Amount
0,HINES,G,January,149700.14
1,TORNAKIAN,M,January,112598.39
2,HEUSEL,J,January,110190.56
3,FITZPATRICK,S,January,72128.93
4,BOWERS,R,January,37935.56
...,...,...,...,...
453,LOWE,K,December,11348.34
454,MACHART,D,December,10626.26
455,KELOUGH,N,December,10253.47
456,BALLARD,J,December,10244.69


In [29]:
print(monthly_spending_over_10000['Amount'].sum())
print(monthly_spending_over_10000.shape)
print(monthly_spending_over_10000.dtypes)

11151603.42
(458, 4)
CardholderLastName         object
CardholderFirstInitial     object
Month                      object
Amount                    float64
dtype: object


#### 4. Filter for purchases over $5000 that were potentially split into multiple transactions (Same purchaser, vendor, and transaction date)

In [30]:
# Group by CardholderLastName, CardholderFirstInitial, Vendor, and TransactionDate
grouped_transactions = (
    pcard_filtered.groupby(['CardholderLastName', 'CardholderFirstInitial', 'Vendor', 'TransactionDate']).agg(
        PostedDate=('PostedDate', 'first'),
        Amount=('Amount', 'sum'),
        Description=('Description', 'first'), 
        MCC=('MCC', 'first'),
        TransactionCount=('Amount', 'size'),
    )
)

In [31]:
# Filter groups where total spending exceed $5,000
split_transactions = grouped_transactions[grouped_transactions['Amount'] > 5000]
split_transactions.reset_index(inplace=True)

In [32]:
# Sort by TransactionDate in ascending order
split_transactions = split_transactions.sort_values('TransactionDate').reset_index(drop=True)
split_transactions = split_transactions[['CardholderLastName', 'CardholderFirstInitial', 'Description', 'Amount', 'Vendor', 'TransactionDate', 'PostedDate', 'MCC', 'TransactionCount']]
split_transactions

,CardholderLastName,CardholderFirstInitial,Description,Amount,Vendor,TransactionDate,PostedDate,MCC,TransactionCount
0,TORNAKIAN,M,HVAC NMB,5164.00,R.B. AKINS COMPANY,2014-01-06,2014-01-08,"CABLE, SATELLITE, AND OTHER PAY TELEVISION AND...",4
1,TORNAKIAN,M,GENERAL PURCHASE,6063.30,AUTOMATED BUILDNG SYSTEMS,2014-01-08,2014-01-09,"HEATING,PLUMBING,AND AIR CONDITIONING CONTRACTORS",3
2,HINES,G,GENERAL PURCHASE,5130.00,OKLAHOMA CONTRACTORS SUP,2014-01-16,2014-01-20,CONSTRUCTION MATERIALS NOT ELSEWHERE CLASSIFIED,3
3,HEUSEL,J,ELECTRICAL PARTS EA,5775.10,ANIXTER-115687,2014-01-23,2014-01-24,ELECTRICAL PARTS AND EQUIPMENT,2
4,CLARK,C,GENERAL PURCHASE,12600.00,TLC PAINTING,2014-01-24,2014-01-27,SPECIAL TRADE CONTRACTORS--NOT ELSEWHERE CLASS...,1
...,...,...,...,...,...,...,...,...,...
92,WOOD,C,GENERAL PURCHASE,5296.32,MEDLEY MATERIAL HANDLING,2014-12-12,2014-12-15,DIRCT MARKETING/DIRCT MARKETERS--NOT ELSEWHERE...,2
93,WILKINSON,T,COMPILOT TVLINK S BUNDLE PCS|USER GUIDE PHONAK...,6214.60,PHONAK HEARING SYS,2014-12-17,2014-12-18,"HEARING AIDS--SALES,SERVICE,AND SUPPLIES",6
94,KINDSCHI,J,GENERAL PURCHASE,6582.50,VETERANS AFFRS DMC,2014-12-22,2014-12-23,GOVERNMENT SERVICES--NOT ELSEWHERE CLASSIFIED,2
95,STOVER,C,GENERAL PURCHASE,19233.28,COCKRELL EYECARE CENTER,2014-12-24,2014-12-26,OPTICIANS AND DISPENSING,1


In [33]:
print(split_transactions['Amount'].sum())
print(split_transactions.shape)
print(split_transactions.dtypes)

806912.18
(97, 9)
CardholderLastName                object
CardholderFirstInitial            object
Description                       object
Amount                           float64
Vendor                            object
TransactionDate           datetime64[ns]
PostedDate                datetime64[ns]
MCC                               object
TransactionCount                   int64
dtype: object


#### 5. Count the number of infractions of Internal Control 4 for each individual

In [34]:
# Count the number of violations per cardholder and stores the count in a new column ViolationCount
num_violation = split_transactions.groupby(['CardholderLastName', 'CardholderFirstInitial']).size()
num_violation = num_violation.reset_index()
individual_split_violation_count = num_violation.rename(columns={0: 'ViolationCount'})

In [35]:
# Sort by ViolationCount in descending order
individual_split_violation_count = individual_split_violation_count.sort_values('ViolationCount', ascending=False).reset_index(drop=True)
individual_split_violation_count

,CardholderLastName,CardholderFirstInitial,ViolationCount
0,HINES,G,18
1,STOVER,C,15
2,ROPERS,A,12
3,TORNAKIAN,M,11
4,KINDSCHI,J,8
5,CLARK,C,4
6,HEUSEL,J,2
7,GOTCHER,M,2
8,BAILEY,J,1
9,MOREY,A,1


In [36]:
print(individual_split_violation_count.shape)
print(individual_split_violation_count['ViolationCount'].sum())

(33, 3)
97


#### 6. Check for split transactions between two or more cardholders

In [37]:
# Create copy of pcard_filtered to avoid modifying the original dataframe
pcard_filtered_copy = pcard_filtered.copy()

# Create a new column CardholderFull that concatenates CardholderLastName and CardholderFirstInitial
pcard_filtered_copy['CardholderFull'] = pcard_filtered_copy['CardholderLastName'] + ' ' + pcard_filtered_copy['CardholderFirstInitial']

# Group by Vendor and TransactionDate and find total spending and number of unique cardholders per day
vendor_daily_spending = (
    pcard_filtered_copy.groupby(['Vendor', 'TransactionDate']).agg(
        PostedDate=('PostedDate', 'first'),
        Amount=('Amount', 'sum'), # total spending per day per vendor
        Description=('Description', lambda x: list(x.unique())), 
        MCC=('MCC', 'first'),
        UniqueCardholders=('CardholderFull', 'nunique'), # number of unique cardholders per day per vendor using full name as identifier
        CardholderNames=('CardholderFull', lambda x: list(x.unique())) # list of unique cardholder names per day per vendor
    ).reset_index()
)

In [38]:
# Filter for vendors with more than 1 unique cardholder per day and total spending exceeds $5000
multiple_potential_violations = vendor_daily_spending[(vendor_daily_spending['UniqueCardholders'] > 1) & (vendor_daily_spending['Amount'] > 5000)]

In [39]:
# Sort by TransactionDate 
multiple_potential_violations = multiple_potential_violations.sort_values('TransactionDate').reset_index(drop=True)
multiple_potential_violations = multiple_potential_violations[['CardholderNames', 'Description', 'Amount', 'Vendor', 'TransactionDate', 'PostedDate', 'MCC', 'UniqueCardholders']]
multiple_potential_violations

,CardholderNames,Description,Amount,Vendor,TransactionDate,PostedDate,MCC,UniqueCardholders
0,"[HINES G, TORNAKIAN M]",[GENERAL PURCHASE],6128.60,EEI,2014-01-03,2014-01-06,"HEATING,PLUMBING,AND AIR CONDITIONING CONTRACTORS",2
1,"[KOEHLER G, CAMPBELL P, ALLEN R]",[GENERAL PURCHASE],5524.16,IVG INVITROGEN CORPORA,2014-01-07,2014-01-08,CATALOG MERCHANTS,3
2,"[VENCILL V, WILLIAMS C, STACY S, LONGAN P]",[GENERAL PURCHASE],7459.00,THE JOURNAL RECORD PUBLIS,2014-01-07,2014-01-08,BUSINESS SERVICES NOT ELSEWHERE CLASSIFIED,4
3,"[ECHOLS CH M, GREGORY I]",[GENERAL PURCHASE],6935.00,AFLV,2014-01-10,2014-01-13,"CIVIC,SOCIAL AND FRATERNAL ASSOCIATIONS",2
4,"[APBLETT A, ARENA A, RICHARDSON J, WHITELEY J,...","[50MM (2) ID CYLINDRICAL H PCE, NORTHWEST ENTE...",5088.41,AMAZON MKTPLACE PMTS,2014-01-14,2014-01-15,BOOK STORES,11
...,...,...,...,...,...,...,...,...
114,"[HOOD K, BURKS S, BRADLEY B, CHANG Y, CLARK M,...","[SANDISK ULTRA 32GB UHI-I/C PCE, LIVESCRIBE 8....",5237.76,AMAZON.COM,2014-12-10,2014-12-10,BOOK STORES,9
115,"[WHITMORE D, WILLIAMS E, CHITWOOD M]",[GENERAL PURCHASE],5327.00,OK DEPT OF CAREER TECH,2014-12-11,2014-12-12,GOVERNMENT SERVICES--NOT ELSEWHERE CLASSIFIED,3
116,"[WHITMORE D, TRAMMELL K]",[GENERAL PURCHASE],6008.15,ESKIMO JOES PROMO PROD,2014-12-12,2014-12-15,"COMMERCIAL PHOTOGRAPHY,ART,AND GRAPHICS",2
117,"[PATAKI M, KINCAID C]",[GENERAL PURCHASE],6125.00,"SQ GRANNY HAD ONE, INC.",2014-12-12,2014-12-15,CATERERS,2


In [40]:
print(multiple_potential_violations['UniqueCardholders'].sum())
print(multiple_potential_violations.shape)

537
(119, 8)


#### 7. Filter for transactions at hotel, motel, resort, or inn MCC that include food or restaurant MCC on the same day

In [41]:
# Filter for accommodation-related transactions
travel_transactions = pcard_filtered[
    pcard_filtered['MCC'].str.contains('HOTEL|MOTEL|RESORT|INN', case=False, na=False)
]

# Filter for food-related transactions
food_transactions = pcard_filtered[
    pcard_filtered['MCC'].str.contains('FOOD|RESTAURANT', case=False, na=False)
]

In [42]:
# Group the accommodation transactions by CardholderLastName, CardholderFirstInitial, and TransactionDate
grouped_travel = travel_transactions.groupby(['CardholderLastName', 'CardholderFirstInitial', 'TransactionDate'])

In [43]:
# Finding matching food transactions for each travel transaction group
results = []
for group, travel_data in grouped_travel:
    # Get cardholder and date from the current group
    last_name, first_initial, transaction_date = group
    
    # Find corresponding food transactions for the same cardholder and date
    matching_food = food_transactions[
        (food_transactions['CardholderLastName'] == last_name) &
        (food_transactions['CardholderFirstInitial'] == first_initial) &
        (food_transactions['TransactionDate'] == transaction_date)
    ]
    
    # Append matching transactions to results
    if not matching_food.empty:
        results.append(matching_food)
        results.append(travel_data)

In [44]:
# Combining the results to a single DataFrame
travel_food_violations = pd.concat(results, ignore_index=True)

In [45]:
# Sort by CardholderLastName and TransactionDate
travel_food_violations = travel_food_violations.sort_values(['CardholderLastName', 'Amount']).reset_index(drop=True)
travel_food_violations = travel_food_violations[['CardholderLastName', 'CardholderFirstInitial', 'Description', 'Amount', 'Vendor', 'TransactionDate', 'PostedDate', 'MCC']]
travel_food_violations

,CardholderLastName,CardholderFirstInitial,Description,Amount,Vendor,TransactionDate,PostedDate,MCC
0,BAKER,B,GENERAL PURCHASE,6.67,CHICK-FIL-A #01973,2014-03-08,2014-03-10,FAST FOOD RESTAURANTS
1,BAKER,B,GENERAL PURCHASE,15.00,JIMMY JOHNS # 563,2014-03-08,2014-03-10,FAST FOOD RESTAURANTS
2,BAKER,B,GENERAL PURCHASE,35.00,MCALISTER'S DELI 710,2014-03-08,2014-03-10,FAST FOOD RESTAURANTS
3,BAKER,B,ROOM CHARGES,103.95,MICROTEL INN,2014-03-08,2014-03-10,MICROTEL INN AND SUITES
4,BAKER,B,ROOM CHARGES,103.95,MICROTEL INN,2014-03-08,2014-03-10,MICROTEL INN AND SUITES
...,...,...,...,...,...,...,...,...
241,YOUNG,C,ROOM CHARGES,203.39,HOLIDAY INN EXPRESS,2014-04-13,2014-04-16,HOLIDAY INNS
242,YOUNG,C,GENERAL PURCHASE,270.82,BJS RESTAURANTS 471,2014-04-13,2014-04-14,EATING PLACES AND RESTAURANTS
243,YOUNG,C,ROOM CHARGES,551.64,FAIRFIELD INN&SUITES MONT,2014-02-23,2014-02-25,FAIRFIELD INN
244,YOUNG,C,ROOM CHARGES,551.64,FAIRFIELD INN&SUITES MONT,2014-02-23,2014-02-25,FAIRFIELD INN


### Exporting results to separate file

In [46]:
# Dictionary of internal control dataframes
controls = {
    'Control 1': transactions_over_5000,
    'Control 2': spending_over_50000,
    'Control 3': monthly_spending_over_10000,
    'Control 4': split_transactions,
    'Control 5': individual_split_violation_count,
    'Control 6': multiple_potential_violations,
    'Control 7': travel_food_violations
}

output_file = 'internal_controls.xlsx'

# Export each control to a separate sheet in an Excel file
with pd.ExcelWriter(output_file, engine='xlsxwriter') as writer:
    for control_name, control_df in controls.items():
        control_df.to_excel(writer, sheet_name=control_name, index=False)